In [1]:
from pre5 import data_pre
from input_data import input_df
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
df = input_df()
df = data_pre(df)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
df_test = pd.read_csv("/tmp/working/dataset/nishika/old_apartment_2020/test.csv", index_col=0)

df_test = data_pre(df_test)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 651975 entries, 1117225 to 47008266
Data columns (total 22 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   都道府県名         651975 non-null  category
 1   市区町村名         651975 non-null  category
 2   地区名           651318 non-null  category
 3   最寄駅：名称        649342 non-null  category
 4   最寄駅：距離（分）     628916 non-null  float64 
 5   間取り           629546 non-null  category
 6   面積（㎡）         651975 non-null  float64 
 7   建築年           633280 non-null  float64 
 8   建物の構造         637098 non-null  category
 9   用途            601333 non-null  category
 10  今後の利用目的       288948 non-null  category
 11  都市計画          633162 non-null  category
 12  建ぺい率（％）       629168 non-null  float64 
 13  容積率（％）        629168 non-null  float64 
 14  取引時点          651975 non-null  float64 
 15  改装            593301 non-null  category
 16  取引の事情等        18397 non-null   category
 17  取引価格（総額）_log  651975 

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae
import lightgbm
import optuna.integration.lightgbm as lgb


df_train, df_val = train_test_split(df, test_size=0.2)

col = "取引価格（総額）_log"
train_y = df_train[col]
train_x = df_train.drop(col, axis=1)

val_y = df_val[col]
val_x = df_val.drop(col, axis=1)

trains = lgb.Dataset(train_x, train_y)
valids = lgb.Dataset(val_x, val_y)

In [6]:
opt_params = {
    "objective":"regression",
    "metric":"mae"
}

import time
start = time.time()
opt=lgb.train(
    opt_params,
    trains,
    valid_sets = valids,
    verbose_eval=200,
    num_boost_round = 600,
    early_stopping_rounds = 100,
)
end = time.time()
print('elaptedtime:', end-start)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.127947 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12615
[LightGBM] [Info] Number of data points in the train set: 521580, number of used features: 21


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] Start training from score 7.219301
Training until validation scores don't improve for 100 rounds
[200]	valid_0's l1: 0.0781661
[400]	valid_0's l1: 0.0765384
[600]	valid_0's l1: 0.0761282
Did not meet early stopping. Best iteration is:
[599]	valid_0's l1: 0.0761273
elaptedtime: 67.41884922981262


In [7]:
# lightGBM
import lightgbm as lgb
best_params = opt.params
model = lgb.train(best_params, trains, valid_sets=valids, num_boost_round=600, early_stopping_rounds=100, verbose_eval=200)

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.097360 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12615
[LightGBM] [Info] Number of data points in the train set: 521580, number of used features: 21
[LightGBM] [Info] Start training from score 7.219301
Training until validation scores don't improve for 100 rounds
[200]	valid_0's l1: 0.0781661
[400]	valid_0's l1: 0.0765384
[600]	valid_0's l1: 0.0761282
Did not meet early stopping. Best iteration is:
[599]	valid_0's l1: 0.0761273


In [8]:
vals = model.predict(val_x)
mae(vals, val_y)

0.07612732996829302

In [9]:
pd.DataFrame(model.feature_importance(), index=val_x.columns, columns=["importance"]).sort_values("importance", ascending=False)

,importance
地区名,5818
最寄駅：名称,3594
建築年,1637
市区町村名,1559
取引時点,1235
面積（㎡）,1068
最寄駅：距離（分）,792
間取り,405
都市距離,260
改装,239


In [10]:
predict = model.predict(df_test)
df_test["取引価格（総額）_log"] = predict
df_test[["取引価格（総額）_log"]].to_csv("./output/12_sub.csv")

In [18]:
df.head()

,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,...,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log,緯度,経度,クラスタ平均,都市距離,平均価格
ID,,,,,,,,,,,,,,,,,,,,,
1117225,北海道,苫小牧市,新中野町,苫小牧,25.0,３ＬＤＫ,120.0,12.0,ＲＣ,住宅,...,200.0,2018.75,未改装,NaN,7.431364,42.631246,141.603122,2.0,60.455317,0.061928
1114724,北海道,苫小牧市,青葉町,青葉,9.0,３ＬＤＫ,70.0,35.0,ＲＣ,住宅,...,200.0,2017.75,未改装,NaN,6.342423,42.631246,141.603122,2.0,60.455317,0.090606
1114753,北海道,苫小牧市,青葉町,青葉,3.0,３ＬＤＫ,70.0,31.0,ＲＣ,住宅,...,200.0,2016.50,改装済,NaN,6.778151,42.631246,141.603122,2.0,60.455317,0.096831
1120806,北海道,苫小牧市,表町,苫小牧,8.0,４ＬＤＫ,105.0,18.0,ＲＣ,住宅,...,400.0,2011.99,未改装,NaN,7.230449,42.631246,141.603122,2.0,60.455317,0.068861
1117264,北海道,苫小牧市,新中野町,苫小牧,25.0,３ＬＤＫ,90.0,15.0,ＲＣ,住宅,...,200.0,2015.50,未改装,NaN,7.204120,42.631246,141.603122,2.0,60.455317,0.080046
